In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  2 16:11:12 2019

@author: colinlaurence
"""
import geopandas as gp
import pandas as pd
import numpy as np
import fiona
import shapely
import datetime as dt
import matplotlib as plt
#import cartopy
import pyproj
from scipy.spatial import distance
import time

In [31]:
stops_df = pd.read_csv("vta_gtfs/stops.txt",header=0)
nodes_df = pd.read_csv("mobiliti_data/nodes_s_c.csv",header=0)

In [3]:



node_lat=nodes_df.lat[0]
node_lon=nodes_df.lng[0]
#print(node_lat)
def wgs2utm(row):
    myProj = pyproj.Proj("+proj=utm +zone=10S, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    xy=myProj(row['lng'], row['lat'], inverse=False)
    return np.array(xy)
    


#nodes_df['xy']= nodes_df.apply(wgs2utm,axis=1)
nodesxy=np.stack(nodes_df.apply(wgs2utm,axis=1))

stops_df.rename(columns={'stop_lat':'lat', 'stop_lon':'lng'}, inplace=True)
#stops_df['xy']= stops_df.apply(wgs2utm,axis=1)
stopsxy=np.stack(stops_df.apply(wgs2utm,axis=1))




    


In [6]:
dists=distance.cdist(stopsxy,nodesxy, 'euclidean')
processdone = time.time()
print(dists)

[[10022.97295623 10182.79696135 17861.86815733 ... 25787.21647456
  31346.79315565 26120.67236703]
 [11202.13545868 11283.78799431 14394.245415   ... 20616.44055495
  25323.45439721 28501.2701006 ]
 [ 9506.19515944  9586.45262233 16075.69150214 ... 19928.25666217
  24994.09061191 26802.21903069]
 ...
 [41645.04053037 41817.30880068 36700.81310993 ... 58886.76619891
  64087.05361097 51697.33395701]
 [41536.3797368  41708.6461375  36617.31633588 ... 58780.6071162
  63983.00537015 51592.24082486]
 [41303.2951228  41475.57305158 36414.23411043 ... 58545.91430936
  63749.86904475 51378.96243685]]


NameError: name 'setupdone' is not defined

In [7]:
closest_center=np.array([list(np.where(item == item.min())[0])[0] for item in dists])



In [10]:
closest_center[0:5]

array([129472,   1130,   1928, 138352,  32755])

In [11]:
nodesxy[0:5]

array([[  606649.62326023, 14131732.51566698],
       [  606571.44977783, 14131886.86947583],
       [  594482.78916395, 14109372.80670198],
       [  594559.42966556, 14109272.69700111],
       [  590122.5301304 , 14113508.01765876]])

In [13]:
stopsxy[0:5]

array([[  607374.74622821, 14121735.8070305 ],
       [  600165.22577357, 14122597.94243833],
       [  601066.82184526, 14124038.35718341],
       [  600806.74573596, 14125863.62665747],
       [  598866.12027832, 14129985.51739765]])

In [15]:
nodes_df[0:5]

,Unnamed: 0,id,x,y,lat,lng,taz_id,geometry
0,249,1013078525,606648.9942,4131732.508,37.32614,-121.79623,27341.0,POINT (-121.79623 37.32614)
1,250,1013078546,606570.9606,4131887.707,37.32754,-121.79709,27341.0,POINT (-121.79709 37.32754)
2,268,1006462339,594482.5871,4109373.392,37.12594,-121.93637,27854.0,POINT (-121.93637 37.12594)
3,269,1006462353,594559.0048,4109273.347,37.12503,-121.93552,27854.0,POINT (-121.93552 37.12503)
4,270,1006483322,590121.6858,4113508.918,37.16364,-121.98495,27854.0,POINT (-121.98495 37.16364)


In [17]:
stops_df[0:5]

,stop_id,stop_code,stop_name,stop_desc,lat,lng,zone_id
0,4785,64785,SANTA TERESA STATION (1),NaN,37.235966,-121.789491,1
1,4735,64735,ALMADEN STATION (1),NaN,37.244539,-121.870638,1
2,4733,64733,OHLONE-CHYNOWETH STATION (1),NaN,37.257423,-121.860279,1
3,4780,64780,CAPITOL STATION (1),NaN,37.273901,-121.862964,1
4,4778,64778,TAMIEN STATION (1),NaN,37.311257,-121.884300,1


In [18]:
stopsxy[0]

array([  607374.74622821, 14121735.8070305 ])

In [19]:
nodesxy[129472]

array([  607344.6257142 , 14121600.45829421])

In [21]:
stopsxy[1]


array([  600165.22577357, 14122597.94243833])

In [30]:
nearest_node_id=[nodes_df['id'][i] for i in closest_center]

In [32]:
node_id=pd.DataFrame(nearest_node_id,columns=['node_id'])

,index
0,129472
1,1130
2,1928
3,138352
4,32755


In [33]:
#near_pnts=[np.where(closest_center==i) for i in range(self.k)]
#new_centers=[np.mean(kmeans_obj.X[near_pnts[i]],axis=0) for i in range(self.k)]
stops_df.assign(node_id=node_id.values)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,node_id
0,4785,64785,SANTA TERESA STATION (1),NaN,37.235966,-121.789491,1,312411624
1,4735,64735,ALMADEN STATION (1),NaN,37.244539,-121.870638,1,1071874080
2,4733,64733,OHLONE-CHYNOWETH STATION (1),NaN,37.257423,-121.860279,1,1101343716
3,4780,64780,CAPITOL STATION (1),NaN,37.273901,-121.862964,1,1760545315
4,4778,64778,TAMIEN STATION (1),NaN,37.311257,-121.884300,1,2701120778
5,4775,64775,CONVENTION CENTER STATION (1),NaN,37.329809,-121.890538,1,538663198
6,4748,64748,SANTA CLARA STATION (0),NaN,37.335924,-121.890354,1,65578415
7,4770,64770,CIVIC CENTER STATION (1),NaN,37.351380,-121.901956,1,2646320715
8,4753,64753,METRO/AIRPORT STATION (0),NaN,37.370364,-121.915990,1,312120468
9,4765,64765,BONAVENTURA STATION (1),NaN,37.387197,-121.928715,1,1700445693


In [34]:
stops_df.to_csv(r'VTA_stops_with_node_ids.csv')